# Search

In this homework, you'll implement a basic search engine by defining your own Python classes. A **search engine** is an algorithm that takes a query and retrieves the most relevant documents for that query. In order to identify the most relevant documents, our search engine will use **term frequency–inverse document frequency** ([tf–idf](https://en.wikipedia.org/wiki/Tf%E2%80%93idf)), an information statistic for determining the relevance of a term to each document from a corpus consisting of many documents.

The **tf–idf statistic** is a product of two values: term frequency and inverse document frequency. **Term frequency** computes the number of times that a term appears in a **document** (such as a single Wikipedia page). If we were to use only the term frequency in determining the relevance of a term to each document, then our search result might not be helpful since most documents contain many common words such as "the" or "a". In order to downweight these common terms, the **document frequency** computes the number of times that a term appears across the **corpus** of all documents.

You will need the `doggos` and `small_wiki` directories to be copied to your work area in the same folder. You do not need to submit these folders but should submit this notebook file after `Run All` and saved.

In [7]:
# You must pass all mypy strict syntax checks

%pip install -q nb_mypy
%reload_ext nb_mypy
%nb_mypy mypy-options --strict

Version 1.0.6


Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install -q pytest ipytest

import os
import math
import re

import pytest
import ipytest
ipytest.autoconfig()


def clean(token: str, pattern: re.Pattern[str] = re.compile(r"\W+")) -> str:
    """
    Returns all the characters in the token lowercased and without matches to the given pattern.

    >>> clean("Hello!")
    'hello'
    """
    return pattern.sub("", token.lower())

Note: you may need to restart the kernel to use updated packages.


## Outside Sources

Update the following Markdown cell to include your name and list your outside sources. Submitted work should be consistent with the curriculum and your sources.

**Name**: Aadhya Goyal

1. https://www.programiz.com/python-programming/methods/string/count
2. https://www.w3schools.com/python/python_sets.asp
3. getting all the values in a dictionary: https://www.geeksforgeeks.org/python/how-can-to-get-list-of-values-from-dictionary-in-python/
4. Python sets: https://www.w3schools.com/python/python_sets.asp
5. formatting string: https://www.w3schools.com/python/python_string_formatting.asp 
6. https://stackoverflow.com/questions/59390797/how-to-type-as-a-string-in-python 
7. merging without duplicates: https://www.geeksforgeeks.org/python/python-merge-two-lists-without-duplicates/ 
8. https://stackoverflow.com/questions/613183/how-do-i-sort-a-dictionary-by-value 
9. dictionary comphrehensions: https://www.geeksforgeeks.org/python/python-dictionary-comprehension/ 
10. default dict: https://docs.python.org/3/library/collections.html#collections.defaultdict 
11. https://www.w3schools.com/python/python_sets_join.asp 
12. https://www.w3schools.com/python/python_operators_bitwise.asp 


In [16]:
%%ipytest -v
# import time

class Document:
    def __init__(self, path: str) -> None:
        # self.start = time.time()
        # print ("in intialization")
        self.path = path
        self.words: dict[str, int] = {}
        with open (path) as f:
            fileString = f.read()
            # file is a string now

            # First, we split the string
            tokens = fileString.split()

            # now that we have a list of all the words, its time to compile a dictionary
            # in this for loop, we iterate through each word in the list of words
            for token in tokens:
                # first, before doing anything with the token, we need to clean it
                token = clean(token)
                
                # check if the word already exists. 
                if(token in self.words):
                    # this means that the token exists
                    # update frequency without creating a new word
                    self.words[token] += 1
                else :
                    # the token doesn't exist
                    # create token and set frequency to be 1
                    self.words[token] = 1
            
            # print(self.words)
            # print("time to init", time.time() - self.start)
            

    def term_frequency(self, term: str) -> float:
        # print("time", time.time() - self.start)
        return (self.words[clean(term)]/(sum(list(self.words.values()))))
    
    def get_words(self) -> set[str]:
        return set(self.words.keys())

    def get_path(self) -> str:
        return self.path

class TestDocument:
    doc1 = Document("doggos/doc1.txt")
    euro = Document("small_wiki/Euro - Wikipedia.html")
    testCase3 = Document("small_wiki/1964 Republican National Convention - Wikipedia.html")

    def test_term_frequency(self) -> None:
        assert self.doc1.term_frequency("dogs") == pytest.approx(1 / 5)
        assert self.euro.term_frequency("Euro") == pytest.approx(0.0086340569495348)
        # assert 

    def test_get_words(self) -> None:
        assert self.doc1.get_words() == set("dogs are the greatest pets".split())
        assert set(w for w in self.euro.get_words() if len(w) == 1) == set([
            *"0123456789acefghijklmnopqrstuvxyz".lower() # All one-letter words in Euro
        ])

    # def test_get_path(self) -> None:
    #     assert self.doc1.getPath("dogs") == pytest.approx(1 / 5)
        



======================================= test session starts =======================================
platform win32 -- Python 3.11.4, pytest-7.4.0, pluggy-1.0.0
rootdir: c:\Users\AadhyaGoyal\Downloads\CodingProjects\cse163-nchsAadhyaGoyal\homework
plugins: anyio-3.5.0
collected 2 items

t_8c9af6cf71b64181b04090cc5aa0a9cf.py ..                                                     [100%]

======================================== 2 passed in 0.03s ========================================


## Task: `SearchEngine`

Write and test a `SearchEngine` class in the code cell below that represents a corpus of `Document` objects and includes methods to compute the tf–idf statistic between a given query and every document in the corpus. The `SearchEngine` begins by constructing an **inverted index** that associates each term in the corpus to the list of `Document` objects that contain the term.

To iterate over all the files in a directory, call `os.listdir` to list all the file names and join the directory to the filename with `os.path.join`. The example below will print only the `.txt` files in the `doggos` directory.

In [10]:
path = "doggos"
extension = ".txt"
for filename in os.listdir(path):
    if filename.endswith(extension):
        print(os.path.join(path, filename))

doggos\doc1.txt
doggos\doc2.txt
doggos\doc3.txt


The `SearchEngine` class should include:

1. An initializer that takes a `str` path to a directory such as `"small_wiki"` and a `str` file extension and constructs an inverted index from the files in the specified directory matching the given extension. By default, the extension should be `".txt"`. Assume the string represents a valid directory, and that the directory contains only valid files. Do not recreate any behavior that is already done in the `Document` class—call the `get_words()` method! Create at most one `Document` per file.

1. A method `_calculate_idf` that takes a `str` term and returns the inverse document frequency of that term. If the term is not in the corpus, return 0. Inverse document frequency is defined by calling `math.log` on *the total number of documents in the corpus* divided by *the number of documents containing the given term*.

1. A method `__repr__` that returns a string representation of this search engine in the format `SearchEngine('{path}')` (with literal single quotes in the output) where `{path}` is the path to the directory specified in the initializer.

1. A method `search` that takes a `str` **query** consisting of one or more terms and returns a `list` of relevant document paths that match at least one of the cleaned terms sorted by descending tf–idf statistic: the product of the term frequency and inverse document frequency. If there are no matching documents, return an empty list.

**For each of the 3 methods (excluding the initializer) in the `SearchEngine` class, write a testing function that contains at least 3 `pytest`-style assertions based on your own testing corpus and written as expressions like 1 / 5 that show your thinking**. Test cases for `SearchEngine.__repr__` may use the given corpuses `doggos` and `small_wiki` in addition to your corpus. Documentation strings are optional for testing functions.

- The type checker can be too picky about the `sorted` `key` parameter: a type error here can be safely ignored if the code works as intended.

In [17]:
%%ipytest -v
from collections import defaultdict

class SearchEngine:
    def __init__(self, path: str, fileExtension: str = ".txt") -> None:
        self.path = path #the path of the file
        self.documents = set() #all the documents that have that ending

        self.IDI:{str, [Document]} = {} #a dictionary of the words and the document objects where they can be found
        
        for filename in os.listdir(path):
            # this filters only for the specific file extension
            if filename.endswith(fileExtension):
                # then we create a new document object and add it to the set
                self.documents.add(Document(os.path.join(path, filename)))
        
        # now that all the files have been stored into documents, we need to construct the inverted index
        

        # lets start by putting all the words and docs into a dictionary
        docsAndWords = {doc: doc.get_words() for doc in self.documents}

        # for doc, words in docsAndWords.items(): #this gets both the key (document), and value (the words inside it) as a tuple
        #     for word in words: 
        #         if word not in self.IDI:
        #             self.IDI[word] = [] #create empty list if list is null (word hasn't been looked at yet)
                
        #         for doc2, words2 in docsAndWords.items(): #does the same thing as the outer loop
        #             # this part of the function checks if the doc hasn't been added yet and adds if no
        #             if word in words2 and doc2 not in self.IDI[word]:
        #                 self.IDI[word].append(doc2)
        self.IDI = defaultdict(set)  # set avoids duplicates, creates empty set if key doens't exist

        for doc, words in docsAndWords.items():
            for word in set(words):  # remove duplicates per doc
                self.IDI[word].add(doc)  # fast append + no duplicates

                

        

    # this calculates the IDF for the term
    def _calculate_idf(self, term: str) -> float:
    
        # the if statement makes sure that there aren't any errors due to 0
        if((len(self.IDI[term]) == 0) or (len(self.documents) == 0)):
            return 0
        
        # calculate idf by log(dividing the two)    
        return math.log(len(self.documents)/len(self.IDI[term]))


    def __repr__(self) -> str:
        return f"SearchEngine('{{self.path}}')"
    
    def search(self, query: str) -> list[str]:
        # take the query and split it into a list of words
        
        # Get the documents for each word and then put them together into one big set of documents
        allDocs = set()
        for word in query.split():
            allDocs |= self.IDI.get(word, set())
    
        # templist: List[Document] = []
        # for word in searchWords:
        #     # get the docs in the dictionary
        #     tempList.append(self.IDI[word])
        
        docsDictionary = {}

        # then, for each document, calculate the tf-df for each word and add it up
        for doc in set(allDocs):
            score = 0
            for word in query.split():
                score += doc.term_frequency(word) * self._calculate_idf(word)
            docsDictionary[doc.get_path()] = score
            

        # now we need to sort the dictionary
        return [doc_path for doc_path, score in sorted(docsDictionary.items(), key=lambda item: item[1], reverse=True)]
    


        


class TestSearchEngine:
    doggos = SearchEngine("doggos")
    small_wiki = SearchEngine("small_wiki", ".html")
    

    def test_search(self) -> None:
        assert self.doggos.search("love") == ["doggos/doc3.txt"]
        assert self.doggos.search("dogs") == ["doggos/doc3.txt", "doggos/doc1.txt"]
        assert self.doggos.search("cats") == ["doggos/doc2.txt"]
        assert self.doggos.search("love dogs") == ["doggos/doc3.txt", "doggos/doc1.txt"]
        assert self.small_wiki.search("data")[:10] == [
            "small_wiki/Internet privacy - Wikipedia.html",
            "small_wiki/Machine learning - Wikipedia.html",
            "small_wiki/Bloomberg L.P. - Wikipedia.html",
            "small_wiki/Waze - Wikipedia.html",
            "small_wiki/Digital object identifier - Wikipedia.html",
            "small_wiki/Chief financial officer - Wikipedia.html",
            "small_wiki/UNCF - Wikipedia.html",
            "small_wiki/Jackson 5 Christmas Album - Wikipedia.html",
            "small_wiki/KING-FM - Wikipedia.html",
            "small_wiki/The News-Times - Wikipedia.html",
        ]
        

    

<cell>8: error: Invalid type comment or annotation  [valid-type]


======================================= test session starts =======================================
platform win32 -- Python 3.11.4, pytest-7.4.0, pluggy-1.0.0
rootdir: c:\Users\AadhyaGoyal\Downloads\CodingProjects\cse163-nchsAadhyaGoyal\homework
plugins: anyio-3.5.0
collected 1 item

t_8c9af6cf71b64181b04090cc5aa0a9cf.py F                                                      [100%]

============================================ FAILURES =============================================
__________________________________ TestSearchEngine.test_search ___________________________________

self = <__main__.TestSearchEngine object at 0x0000021E0E462090>

    def test_search(self) -> None:
>       assert self.doggos.search("love") == ["doggos/doc3.txt"]
E       AssertionError: assert ['doggos\\doc3.txt'] == ['doggos/doc3.txt']
E         At index 0 diff: 'doggos\\doc3.txt' != 'doggos/doc3.txt'
E         Use -v to get more diff

C:\Users\AadhyaGoyal\AppData\Local\Temp\ipykernel_16620\2516194682.py:

We recommend the following iterative software development approach to implement the `search` method.

1. Write code to handle queries that contain only a single term by collecting all the documents that contain the given term, computing the tf–idf statistic for each document, and returning the list of document paths sorted by descending tf–idf statistic.

1. Write tests to ensure that your program works on single-term queries.

1. Write code to handle queries that contain more than one term by returning all the documents that match any of the terms in the query sorted by descending tf–idf statistic. The tf–idf statistic for a document that matches more than one term is defined as the sum of its constituent single-term tf–idf statistics.

1. Write tests to ensure that your program works on multi-term queries.

Here's a walkthrough of the `search` function from beginning to end. Say we have a corpus in a directory called `"doggos"` containing 3 documents with the following contents:

- `doggos/doc1.txt` with the text `Dogs are the greatest pets.`
- `doggos/doc2.txt` with the text `Cats seem pretty okay`
- `doggos/doc3.txt` with the text `I love dogs!`

The initializer should construct the following inverted index.

```python
{"dogs":     [doc1, doc3],
 "are":      [doc1],
 "the":      [doc1],
 "greatest": [doc1],
 "pets":     [doc1],
 "cats":     [doc2],
 "seem":     [doc2],
 "pretty":   [doc2],
 "okay":     [doc2],
 "i":        [doc3],
 "love":     [doc3]}
```

Searching this corpus for the multi-term query `"love dogs"` should return a list `["doggos/doc3.txt", "doggos/doc1.txt"]` by:

1. Finding all documents that match at least one query term. The word `"love"` is found in `doc3` while the word `"dogs"` is found in `doc1` and `doc3`.

1. Computing the tf–idf statistic for each matching document. For each matching document, the tf–idf statistic for a multi-word query `"love dogs"` is the sum of the tf–idf statistics for `"love"` and `"dogs"` individually.

   1. For `doc1`, the sum of 0 + 0.081 = 0.081. The tf–idf statistic for `"love"` is 0 because the term is not in `doc1`.

   1. For `doc3`, the sum of 0.366 + 0.135 = 0.501.

1. Returning the matching document paths sorted by descending tf–idf statistic.

After completing your `SearchEngine`, run the following cell to search our small Wikipedia corpus for the query "data". Try some other search queries too!

In [12]:
SearchEngine("small_wiki", ".html").search("data")

NameError: name 'defaultdict' is not defined